# datalake

> Interface to datalake in AWS S3

In [ ]:
#| default_exp lake

In [ ]:
#| exporti

import boto3
import json
import yaml
from pathlib import Path
from io import BytesIO
import zipfile
import pandas as pd
import datetime

### Lake Interface class

In [ ]:
#| exporti

class S3ObjectNotFound(Exception):
    pass


class LakeConfigNotFound(Exception):
    pass


class LakeInterface():

    def __init__(
        self,
        config
    ):
        if type(config) == str:
            # load from local config file
            try:
                conf = yaml.safe_load(Path(f'{Path.home()}/.bankdata_config.yml').read_text())
                config_name = config
                config = conf['lake_interface'].get(config_name)
                if config is None:
                    raise LakeConfigNotFound(f'config {config_name} not in Lake config')
            except Exception as e:
                raise LakeConfigNotFound(e)

        session = boto3.session.Session(profile_name=config.get('profile', 'default'))
        self.session=session
        self.s3 = session.client('s3')
        self.athena = session.client('athena')
        self.queries = []
        self.bucket = config.get('default_bucket')
        self.query_results_location = f's3://{self.bucket}/{config.get("query_results_location", "athena_results")}'
        self.athena_workgroup = config.get('athena_workgroup', 'primary')


    def get_object(self, key, bucket=None):
        try:
            return self.s3.get_object(Bucket=self.bucket or DEFAULT_BUCKET, Key=key)
        except Exception as e:
            if e.response['Error']['Code'] == 'NoSuchKey':
                raise S3ObjectNotFound('No S3 object with key = %s' % key)
            else:
                raise

    def load_csv(self,key,bucket=None, delimiter=',', skiprows=None, line_terminator=None):
        obj = self.get_object(key, bucket=bucket or self.bucket)
        if line_terminator:
            return pd.read_csv(obj['Body'], delimiter=delimiter, skiprows=skiprows, lineterminator=line_terminator)
        else:
            return pd.read_csv(obj['Body'], delimiter=delimiter, skiprows=skiprows)
    
    
    def load_json(self, key, output_format='json'):
        obj = self.get_object(key)
        return json.loads(obj['Body'].read())
        
    
    def list_objects(self, prefix, bucket=None):
        
        paginator = self.s3.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket or self.bucket, Prefix=prefix)

        return sum([[obj['Key'] for obj in page['Contents']] for page in pages], [])
    

    def save_json(self, path, data, bucket=None, timestamp=None):
        if timestamp:
            key = f'{path}/timestamp={timestamp}/data.json'
        else:
            key = f'{path}/data.json'

        return self.put_object(key, json.dumps(data), bucket=bucket)
        
    def put_object(self, key, data, metadata={}, bucket=None):
        try:
            resp = self.s3.put_object(
                Bucket=bucket or self.bucket,
                Key=key,
                Body=data
            )
            status_code = resp['ResponseMetadata']['HTTPStatusCode']
            if status_code == 200:
                return True
            else:
                raise Exception(f'Unknown error. Status code: {status_code}')
        except Exception as e:
            raise Exception(f'Unknown error in put object for {key}. {str(e)}')

            
    def put(self, path, df, bucket=None, timestamp=None):
        if timestamp:
            key = f'{path}/timestamp={timestamp}/data.parquet'
        else:
            key = f'{path}/data.parquet'

        out_buffer = BytesIO()
        df.to_parquet(
            out_buffer,
            index=True,
            engine='pyarrow',
            compression='gzip',
            allow_truncated_timestamps=True
        )
        if self.put_object(key, out_buffer.getvalue(), bucket=bucket):
            return f'Saved to {key}'
        else:
            return f'Unknown error in save_parquet: {key}'
    
    def most_recent(self, prefix, bucket=None):
        matched_objects = self.list_objects(
            bucket=bucket or self.bucket,
            prefix=prefix
        )
        
        if len(matched_objects) > 1:
            print(f'Multiple objects found for prefix {prefix}')
            return None
        elif len(matched_objects) == 0:
            print(f'No objects found for prefix {prefix}')
            return None
        else:
            return matched_objects[0]

    
    def get(self, path, bucket=None):
        try:
            key = self.most_recent(path, bucket=bucket)
        except Exception as e:
            print(f'No objects found with path: {key}. {e}')
            return None

        resp = self.get_object(key, bucket=bucket)
        return pd.read_parquet(BytesIO(resp['Body'].read()))
                
    def start_query(self, query_def, query_id):
        
        response = self.athena.start_query_execution(
            QueryString=query_def,
            QueryExecutionContext={
                'Database': 'bankdata'
            },
            WorkGroup=self.athena_workgroup,
            ResultConfiguration={"OutputLocation": self.query_results_location}
        )
        
        query_status = response['ResponseMetadata'].get('HTTPStatusCode')
        
        if query_status == 200:
            query_record = {
                'query_def': query_def,
                'query_id': query_id,
                'execution_id': response['QueryExecutionId']
            }
            try:
                query_ids = [r['query_id'] for r in self.queries]
                query_index = query_ids.index(query_id)
                self.queries[query_index] = query_record
            except:
                # query_id is for new query
                self.queries.append(query_record)
                
            response = {
                'status': 'Query Started',
                'execution_id': response['QueryExecutionId'],
                'query_id': query_id
            }
        else:
            response = response['ResponseMetadata']
            response['status'] = 'Query Failed'
        
        return response

    
    def get_query_results(self, query_id):
        query_result_gen = (
            q for q in self.queries
            if q['query_id'] == query_id
        )

        query = next(query_result_gen)
        
        if query:
            results_paginator = self.athena.get_paginator('get_query_results')
            results_iter = results_paginator.paginate(
                QueryExecutionId=query.get('execution_id'),
                PaginationConfig={
                    'PageSize': 1000
                }
            )
            
            data = []
            for rslt_page in results_iter:
                page_data = [[e.get('VarCharValue') for e in row['Data']] for row in rslt_page['ResultSet']['Rows']]
                data.append(page_data)
            
            
            return pd.DataFrame(columns=data[0], data=data[1:])
        else:
            return None



Example of configurations. 

- profile field refers to the name of the aws profile defined in ~/.aws/credentials

In [ ]:

LAKE_CONFIGS = {
    'machinesp': {
        'profile': 'machinesp',
        'default_bucket': 'machinesp-bank-datasets',
        'query_results_location': 'athena_results',
        'athena_workgroup': 'bankdata'
    },
    'local': {
        'profile': 'default',
        'default_bucket': 'machinesp-datasets',
        'query_results_location': 'athena_results',
        'athena_workgroup': 'primary'
    }
}

Initializing lake interface

In [ ]:
li = LakeInterface(
    config=LAKE_CONFIGS['machinesp']
)

### Utilities

Functions for Common tasks

#### Unzipping from S3 to S3

In [ ]:
#| exporti

def unzip(lake_interface, source_file, destination_folder, exclude_pattern=None, include_pattern=None):
    logs = [
        '-' * 30,
        f'Copying from {source_file} to {destination_folder}'
    ]
    
        
    zip_obj = lake_interface.get_object(source_file)
    buffer = BytesIO(zip_obj["Body"].read())

    z = zipfile.ZipFile(buffer)
    file_names = z.namelist()
    filtered_files = file_names.copy()

    if include_pattern:
        filtered_files = [f for f in filtered_files if include_pattern in f]

    if exclude_pattern:
        filtered_files = [f for f in filtered_files if exclude_pattern not in f]

    for filename in filtered_files:
        file_info = z.getinfo(filename)

        lake_interface.s3.upload_fileobj(
            Fileobj=z.open(filename),
            Bucket=lake_interface.bucket,
            Key=f'{destination_folder}/{filename}'
        )

        logs.append(f'Copied {filename}')

    return logs


#### Fetching Glue Data Catalog

In [ ]:
#| exporti

def parse_column_info(cols):
    return [
        {
            'column_name': c['Name'], 
            'column_type': c['Type']
        } for c in cols if '__index_level' not in c['Name']
    ]
    
def parse_table_info(raw_table):
    return {
        'table_name': raw_table['Name'],
        'databse_name': raw_table['DatabaseName'],
        'location': raw_table['StorageDescriptor']['Location'],
        'columns': parse_column_info(raw_table['StorageDescriptor']['Columns'])
    }

In [ ]:
#| export

def get_data_catalog(lake_interface, database_name):
    """
    Fetches catalog for glue database

    Parameters
    ----------
    lake_interface: LakeInterface, required
        instance of class providing access to datalake
    database_name : str, required
        Name of AWS Glue database

    Returns
    -------
    table_columns : list(str)
        List of all columns in every table in form [table_name].[column_name]

    """
    #harvest aws crawler metadata

    session = lake_interface.session
    glue = session.client('glue')

    next_token = ""
    #glue = boto3.client('glue',region_name='us-east-1')
    tables = []

    while True:
        resp = glue.get_tables(DatabaseName=database_name, NextToken=next_token)

        for tbl in resp['TableList']:
            tables.append(parse_table_info(tbl))
        next_token = resp.get('NextToken')
        
        if next_token is None:
            break

    return tables


In [ ]:
li = LakeInterface(config='machinesp')
bankdata_catalog = get_data_catalog(li, 'bankdata')

### Initialize interface to the lake

> can pass a default bucket to be used and AWS profile (locally usually from ~/.aws/credentials)

Lake interface includes function to list objects with a certain S3 location prefix

In [ ]:
li.list_objects(prefix='banks/call_reports/staging')[:5]

['banks/call_reports/staging/',
 'banks/call_reports/staging/FFIEC CDR Call Bulk All Schedules 03312001.zip',
 'banks/call_reports/staging/FFIEC CDR Call Bulk All Schedules 03312002.zip',
 'banks/call_reports/staging/FFIEC CDR Call Bulk All Schedules 03312003.zip',
 'banks/call_reports/staging/FFIEC CDR Call Bulk All Schedules 03312004.zip']

#### Loading raw files

Raw files can be loaded with specific implementations based on the various forms of data

In [ ]:
sched = 'RCA'

df = li.load_csv(
    'banks/call_reports/raw/20210331/FFIEC CDR Call Schedule RCA 03312021.txt', 
    delimiter='\t',
    skiprows=[1]
)

df[~df['RCFD0010'].map(pd.isnull)].head()

,IDRSSD,RCFD0010,RCFD0022,RCFD0070,RCFD0082,RCFD0090,RCON0010,RCON0020,RCON0070,RCON0080,RCON0082,RCON0090,Unnamed: 12
46,12311,8497788.0,856297.0,21919.0,126173.0,7493399.0,8497788.0,271551.0,21919.0,584746.0,126173.0,7493399.0,NaN
110,30810,20156640.0,436232.0,0.0,947713.0,18772695.0,20156640.0,435969.0,0.0,263.0,947713.0,18772695.0,NaN
130,35301,111590000.0,12000.0,58482000.0,3340000.0,49756000.0,54170000.0,12000.0,1062000.0,0.0,3340000.0,49756000.0,NaN
224,60143,14858000.0,993000.0,357000.0,33000.0,13475000.0,14548000.0,530000.0,47000.0,463000.0,33000.0,13475000.0,NaN
239,63069,5874285.0,395888.0,70856.0,51696.0,5355845.0,5874285.0,260420.0,70856.0,135468.0,51696.0,5355845.0,NaN


#### Put objects to lake

The put action saves to S3 in a standard format

In [ ]:
df = df[[c for c in df.columns if 'Unnamed:' not in c]]
li.put(
    'test/put_example',
    df
)

'Saved to test/put_example/data.parquet'

#### Saving json to lake

In [ ]:
d = [{'a': 1, 'b': 2}, {'a': 4, 'c': 2}]

In [ ]:
li.save_json('test/save_json', d)

True

In [ ]:
li.load_json('test/save_json/data.json')

[{'a': 1, 'b': 2}, {'a': 4, 'c': 2}]

#### Get objects



In [ ]:
li.get('test/put_example')

,IDRSSD,RCFD0010,RCFD0022,RCFD0070,RCFD0082,RCFD0090,RCON0010,RCON0020,RCON0070,RCON0080,RCON0082,RCON0090
0,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5022,5518023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5023,5538937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5024,5561001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5025,5574430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Athena queries

In [ ]:
db_name = 'bankdata'
table_name = 'call_report_sched_ci'
query = f"SELECT COUNT(*) from {db_name}.{table_name}"

resp = li.start_query(query, 'count rows in ci')

Interface stores list of Athena queries

In [ ]:
li.queries

[{'query_def': 'SELECT COUNT(*) from bankdata.call_report_sched_ci',
  'query_id': 'count rows in ci',
  'execution_id': '9372cd85-286f-4ae1-bc1b-422789f95485'}]

The query id is used to retrieve the query results

In [ ]:
li.get_query_results('count rows in ci')

,_col0
0,107073
